# TPC-H 100GB Demo Notebook

### Create BlazingSQL Context

In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext(
    pool=True,
    
    config_options={"BLAZING_DEVICE_MEM_RESOURCE_CONSUMPTION_THRESHOLD":.1}
)

### Register S3 Bucket to BlazingSQL

In [ ]:
# register AWS S3 bucket
bc.s3('bsql_data', bucket_name='blazingsql-colab')


### Create Tables for SF100 TPC-H

In [ ]:
%%time
bc.create_table('orders', 's3://bsql_data/tpch_sf100/orders/')

bc.create_table('lineitem', 's3://bsql_data/tpch_sf100/lineitem/')

bc.create_table('customer', 's3://bsql_data/tpch_sf100/customer/')

bc.create_table('part', 's3://bsql_data/tpch_sf100/part/')

bc.create_table('partsupp', 's3://bsql_data/tpch_sf100/partsupp/')

bc.create_table('supplier', 's3://bsql_data/tpch_sf100/supplier/')

bc.create_table('nation', 's3://bsql_data/tpch_sf1/nation/')

bc.create_table('region', 's3://bsql_data/tpch_sf1/region/')

In [ ]:
%%time
# Q3

query = """
    select
        l_orderkey,
        sum(l_extendedprice*(1-l_discount)) as revenue,
        o_orderdate,
        o_shippriority
    from
        customer,
        orders,
        lineitem
    where
        c_mktsegment = 'BUILDING'
        and c_custkey = o_custkey
        and l_orderkey = o_orderkey
        and o_orderdate < date '1995-03-15'
        and l_shipdate > date '1995-03-15'
    group by
        l_orderkey,
        o_orderdate,
        o_shippriority
    order by
        revenue desc,
        o_orderdate
"""
print(bc.explain(query))
bc.sql(query)

In [ ]:
%%time
# Q4
query = """ 
     select
         o_orderpriority, 
         count(*) as order_count
     from
         orders
     where
         exists (
             select
                 *
             from
                 lineitem
             where
                 l_orderkey = o_orderkey
                 and l_commitdate < l_receiptdate
                 )
     group by
         o_orderpriority
     order by
         o_orderpriority
         """
print(bc.explain(query))
bc.sql(query)

In [ ]:
%%time
# Q6
query = """
     select
         sum(l_extendedprice*l_discount) as revenue
     from
         lineitem
     where
         l_shipdate >= date '1994-01-01' 
         and l_shipdate < date '1995-01-01'
         and l_discount between 0.05 and 0.07 
         and l_quantity < 24
         """
# run query
print(bc.explain(query))
bc.sql(query)

In [ ]:
bc.log("select * from bsql_queries where ral_id = 0")